In [9]:
import networkx as nx
import random
import os
import pandas as pd
import multiprocessing
import argparse
from reliability_comp import reliability_comp_fbs, reliability_comp_mc

In [10]:
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--seed", default=1871, type=int, help="seed number")
    parser.add_argument("--n_graph", default=10, type=int, help="number of graphs")
    parser.add_argument("--n_node", default=8, type=int, help="number of nodes")
    parser.add_argument("--l_bound", default=0.3, type=float, help="edge addition lower bound")
    parser.add_argument("--u_bound", default=0.6, type=float, help="edge addition upper bound")
    parser.add_argument("--path", default='reliability_tdzdd/', type=str, help="path to the folder")
    parser.add_argument("--mc_run", default=10000, type=int, help="number of Monte Carlo runs")
    parser.add_argument("--mc_scenario", default=100, type=int, help="number of Monte Carlo scenarios")
    parser.add_argument("--confidence", default=0.95, type=float, help="confidence level")
    parser.add_argument("--comp_type", default=2, type=int, help="type of computation: 1-exact fbs, 2-estimation monte carlo, 3-exact enumeration")
    args = parser.parse_args(args=[])
    return args

In [11]:
args = get_args()

In [19]:
def create_graph(args):

    random.seed(args.seed)

    dataset = []
    number_of_graphs_created = 0

    while number_of_graphs_created < args.n_graph:

        N = args.n_node

        #reliabilty = [0.80, 0.85, 0.90, 0.95, 0.99]
        # Create an empty graph object
        g = nx.Graph()


        # Adding nodes
        g.add_nodes_from(range(0, N))


        # Add edges to the graph randomly.
        for i in g.nodes():
            for j in g.nodes():
                if (i < j):

                    # Take random number R.
                    R = random.random()

                    # Check if R is in the range [0.3, 0.56]
                    if (args.l_bound <= R <= args.u_bound):
                        g.add_edge(i, j)
                        nx.set_edge_attributes(g, {(i, j): {"reliability": random.uniform(0.5,1)}})


        if nx.is_connected(g) == True:
            dataset.append(g)
            number_of_graphs_created += 1
        else:
            continue
    print('Graphs created: ', number_of_graphs_created)
    return dataset

def comp_process(dataset, args):
    #os.chdir(os.path.expanduser('~'))
    os.chdir(args.path)
    #results = []
    if args.comp_type == 1:
        with open ('../save_files/results_{}.csv'.format(args.n_node), 'w') as f:
            for i, t in zip(dataset, range(len(dataset))):
                rel, time = reliability_comp_fbs(i, t, args.n_node)
            f.write('{};{};{}\n'.format(nx.to_dict_of_dicts(i), rel, time))
        f.close()
    
    elif args.comp_type == 2:
        
        rel_args = [(i, args, t, args.n_node) for i,t in zip(dataset, range(len(dataset)))]
        with multiprocessing.Pool(processes=32) as pool:
            results = pool.starmap(reliability_comp_mc, rel_args)

        with open ('../save_files/results_{}.csv'.format(args.n_node), 'w') as f:
            for i, t in zip(dataset, range(len(dataset))):
            #     rel, rel_std, rel_lower, rel_upper, time = reliability_comp_mc(i, args, t, args.n_node)
                f.write('{};{};{};{};{};{}\n'.format(nx.to_dict_of_dicts(i), results[t][0],results[t][1], results[t][2], results[t][3], results[t][4]))
        f.close()


In [22]:
if __name__ == "__main__":
    args = get_args()
    dataset = create_graph(args)
    #print(multi_process(dataset))
    comp_process(dataset, args)

Graphs created:  10


In [21]:
%cd ..

/Users/farid/Downloads/All-Terminal-Reliability


In [10]:
dataset = create_graph(args)

Graphs created:  10


In [ ]:
def confidence_interval(data, confidence=0.95):

    mean = np.mean(data)
    standard_error = stats.sem(data)
    degrees_of_freedom = len(data) - 1
    t_critical = stats.t.ppf((1 + confidence) / 2, degrees_of_freedom)
    margin_of_error = t_critical * standard_error
    
    lower_bound = mean - margin_of_error
    upper_bound = mean + margin_of_error
    
    return lower_bound, upper_bound

In [33]:
for i in dataset:
    data = all_terminal_reliability_mc(i)
    print(np.mean(data), confidence_interval(data)[0], confidence_interval(data)[1])

0.9725800000000001 0.9712792650955182 0.973880734904482
0.17447 0.17211377028465968 0.1768262297153403
0.5211699999999999 0.5180157437918327 0.5243242562081671
0.8009700000000001 0.7973900295764824 0.8045499704235177
0.34133 0.33763960105437335 0.3450203989456267
0.12268000000000001 0.12066573425155694 0.12469426574844308
0.33359000000000005 0.3309608646677505 0.3362191353322496
0.26303 0.2599779875158988 0.2660820124841012
0.5398099999999999 0.5348647211578869 0.5447552788421129
0.3051 0.30200323986637756 0.3081967601336224


In [14]:
from reliability_comp import reliability_comp_fbs

In [21]:
%cd reliability_tdzdd

/Users/farid/Downloads/All-Terminal-Reliability/reliability_tdzdd


In [24]:
for i, g in zip(range(len(dataset)), dataset):
    print(reliability_comp_fbs(g, i, 8)[0])

0.9736604021
0.1745074378
0.5199807424
0.8018757519
0.3396214091
0.1240917568
0.3346791981
0.2652119244
0.5359889512
0.3091974369


In [20]:
!pwd

/Users/farid/Downloads/All-Terminal-Reliability


In [18]:
range(len(dataset))

range(0, 10)